MUTSUI COMPETITION


In [1]:
#importar librerías

import os 
import numpy as np 
import polars as pl
import pandas as pd
import lightgbm as lgb
import xgboost as xgb

In [2]:
#leer datos con polars
train_features = pd.read_csv("mitsui-commodity-prediction-challenge/train.csv")
pairs_df = pd.read_csv("mitsui-commodity-prediction-challenge/target_pairs.csv")
train_labels = pd.read_csv("mitsui-commodity-prediction-challenge/train_labels.csv")





In [3]:
# --- Constants and Global Variables ---
NUM_TARGET_COLUMNS = 424
# This dictionary will hold our 424 trained models
trained_models = {}

Creamos la función que nos va a dar las columnas 

In [4]:
# --- Reusable Feature Engineering Function ---
def create_targeted_features(df, pairs_df):
    """
    Creates targeted features (ratios or moving averages) for all targets.
    This function must be used on both the training and test data to ensure consistency.
    """
    # Optimized to avoid performance warnings and improve speed
    new_features = {} # Use a dictionary to store new columns before adding them
    #esto del diccionario solo sirve para ser más rápido porque luego convertiremos el diccionario en una columna 

   #para el bucle principal vamos a recorrer cada fila de pairs y coger el valor de la columna "pair" para esa fila  
    for i, row in pairs_df.iterrows():
        pair_string = row['pair']
        
        #este if lo vamos a usar paa distiguir si es un par de commodities o una sola commoditie ya que llevan el simbolo -
        if ' - ' in pair_string: # It's a pair of instruments
            instrument_1, instrument_2 = pair_string.split(' - ')
            feature_name = f'ratio_{instrument_1}_vs_{instrument_2}'
            #aqui simplemente separa en commoditie 1 y commoditie 2 mediante la funcion split

            # Ensure columns exist before creating feature, verifica que ambos commodities existen y hace la resta, la suma
            # es para no dividir entre 0
            if instrument_1 in df.columns and instrument_2 in df.columns:
                new_features[feature_name] = df[instrument_1] / (df[instrument_2] + 1e-6)
        else: # It's a single instrument y simplemente hace el moving avarage con la función rolling del single pair
            instrument_1 = pair_string
            feature_name = f'ma_10_{instrument_1}'
            if instrument_1 in df.columns:
                new_features[feature_name] = df[instrument_1].rolling(window=10, min_periods=1).mean()
    # Convert the dictionary of new features to a DataFrame, pasar de diccionario usado a dataframe
    new_features_df = pd.DataFrame(new_features)
    
    # Concatenate the original df with the new features df for better performance
    return pd.concat([df, new_features_df], axis=1)

# FIX: Ensure all feature columns are numeric, coercing errors to NaN
base_features = [col for col in train_features.columns if col != 'date_id']
#esto es una línea que crea una list comprehension que recorre todas las columnas de train_features y
# añade a la lista base_features todas las columnas excepto 'date_id'
for col in base_features:
    train_features[col] = pd.to_numeric(train_features[col], errors='coerce')
    #obliga a cambiar todo a numérico y si no puede lo cambia a NaN

train_df = pd.merge(train_features, train_labels, on='date_id', how='inner')
train_df = train_df.sort_values(by='date_id').reset_index(drop=True)
#aquí simplemente hace un merge de train_features y train_labels usando 'date_id' como clave
# luego ordena el dataframe resultante por 'date_id' y resetea los índices



In [6]:
# 2. Create all targeted features for the training data
train_df_featured = create_targeted_features(train_df, pairs_df)
# Note: Ensure to use the same function on test data later for consistency 
print("Feature engineering on training data complete.")

# 3. Loop through all 424 targets and train a model for each
# Base features are already defined above
for i, row in pairs_df.iterrows():
    current_target = row['target']
    pair_string = row['pair']

    # Identify the specific feature for this target
    if ' - ' in pair_string:
        instrument_1, instrument_2 = pair_string.split(' - ')
        specific_feature_name = f'ratio_{instrument_1}_vs_{instrument_2}'
    else:
        instrument_1 = pair_string
        specific_feature_name = f'ma_10_{instrument_1}'
        
    all_features_for_model = base_features + [specific_feature_name]
    #aqui crea una lista con todas las features que usará el modelo, las base más la específica

    # Prepare data for this specific model
    temp_df = train_df_featured.dropna(subset=[current_target, specific_feature_name])
    X = temp_df[all_features_for_model]
    y = temp_df[current_target]

    #aqui crea una matriz X con todas las features y una matriz y con el target actual

    # Time-series split (no need for a large validation set here, just for early stopping)
    train_size = int(len(X) * 0.9)
    X_train, y_train = X[:train_size], y[:train_size]
    X_val, y_val = X[train_size:], y[train_size:]

    #COMIENZO DE LAS PRUEBAS PARA XGBOOST
    modeloxgb = xgb.XGBRegressor(colsample_bytree=0.5,
                 gamma=0,                 
                 learning_rate=0.1,
                 max_depth=5,
                 min_child_weight=1.5,
                 n_estimators=1000,                                                               
                 reg_alpha=0.1,
                 reg_lambda=0.9,
                 subsample=0.8,
                 seed=1000,
                 n_jobs=-1,
                 verbosity=1,
                 tree_method="gpu_hist",
                 eval_metric="rmse",
                 early_stopping_rounds=50)
    

    # Aquí añadimos eval_set y early_stopping_rounds
    modeloxgb.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=True)
    trained_models[current_target] = modeloxgb
    
    # Opcional: mostrar progreso
    if (i + 1) % 50 == 0:
        print(f"Entrenados {i + 1}/{len(pairs_df)} modelos")

print(f"--- Model training complete. {len(trained_models)} models trained. ---")


"""
Train Model
    lgbm = lgb.LGBMRegressor(objective='regression_l1', random_state=42, n_jobs=-1, n_estimators=500)
    lgbm.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='mae',
        callbacks=[lgb.early_stopping(50, verbose=False)]
    )
    
    trained_models[current_target] = lgbm
"""

print(f"--- Model training complete. {len(trained_models)} models trained. ---")


# Al final, GUARDAR los modelos
import os
os.makedirs('models', exist_ok=True)

for target_name, model in trained_models.items():
    model.save_model(f'models/{target_name}.json')

print(f"✅ {len(trained_models)} modelos guardados")



Feature engineering on training data complete.
[0]	validation_0-rmse:0.01162


C:\Users\ruben\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\xgboost\callback.py:386: UserWarning: [16:21:44] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  self.starting_round = model.num_boosted_rounds()


[1]	validation_0-rmse:0.01163
[2]	validation_0-rmse:0.01165
[3]	validation_0-rmse:0.01169
[4]	validation_0-rmse:0.01168
[5]	validation_0-rmse:0.01169
[6]	validation_0-rmse:0.01172
[7]	validation_0-rmse:0.01174
[8]	validation_0-rmse:0.01177
[9]	validation_0-rmse:0.01181
[10]	validation_0-rmse:0.01181
[11]	validation_0-rmse:0.01181
[12]	validation_0-rmse:0.01181
[13]	validation_0-rmse:0.01193
[14]	validation_0-rmse:0.01197
[15]	validation_0-rmse:0.01199
[16]	validation_0-rmse:0.01200
[17]	validation_0-rmse:0.01208
[18]	validation_0-rmse:0.01209
[19]	validation_0-rmse:0.01214
[20]	validation_0-rmse:0.01215
[21]	validation_0-rmse:0.01216
[22]	validation_0-rmse:0.01220
[23]	validation_0-rmse:0.01220
[24]	validation_0-rmse:0.01220
[25]	validation_0-rmse:0.01221
[26]	validation_0-rmse:0.01227
[27]	validation_0-rmse:0.01228
[28]	validation_0-rmse:0.01228
[29]	validation_0-rmse:0.01222
[30]	validation_0-rmse:0.01225
[31]	validation_0-rmse:0.01229
[32]	validation_0-rmse:0.01229
[33]	validation_0

C:\Users\ruben\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\xgboost\callback.py:386: UserWarning: [16:21:46] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  self.starting_round = model.num_boosted_rounds()


[7]	validation_0-rmse:0.01431
[8]	validation_0-rmse:0.01434
[9]	validation_0-rmse:0.01439
[10]	validation_0-rmse:0.01438
[11]	validation_0-rmse:0.01439
[12]	validation_0-rmse:0.01440
[13]	validation_0-rmse:0.01440
[14]	validation_0-rmse:0.01440
[15]	validation_0-rmse:0.01435
[16]	validation_0-rmse:0.01436
[17]	validation_0-rmse:0.01437
[18]	validation_0-rmse:0.01440
[19]	validation_0-rmse:0.01441
[20]	validation_0-rmse:0.01444
[21]	validation_0-rmse:0.01440
[22]	validation_0-rmse:0.01440
[23]	validation_0-rmse:0.01442
[24]	validation_0-rmse:0.01442
[25]	validation_0-rmse:0.01445
[26]	validation_0-rmse:0.01447
[27]	validation_0-rmse:0.01450
[28]	validation_0-rmse:0.01452
[29]	validation_0-rmse:0.01454
[30]	validation_0-rmse:0.01454
[31]	validation_0-rmse:0.01450
[32]	validation_0-rmse:0.01448
[33]	validation_0-rmse:0.01446
[34]	validation_0-rmse:0.01447
[35]	validation_0-rmse:0.01444
[36]	validation_0-rmse:0.01444
[37]	validation_0-rmse:0.01444
[38]	validation_0-rmse:0.01446
[39]	valida

...

[5]	validation_0-rmse:0.03302
[6]	validation_0-rmse:0.03286
[7]	validation_0-rmse:0.03292
[8]	validation_0-rmse:0.03308
[9]	validation_0-rmse:0.03275
[10]	validation_0-rmse:0.03273
[11]	validation_0-rmse:0.03290
[12]	validation_0-rmse:0.03301
[13]	validation_0-rmse:0.03281
[14]	validation_0-rmse:0.03269
[15]	validation_0-rmse:0.03258
[16]	validation_0-rmse:0.03286
[17]	validation_0-rmse:0.03286
[18]	validation_0-rmse:0.03303
[19]	validation_0-rmse:0.03303
[20]	validation_0-rmse:0.03307
[21]	validation_0-rmse:0.03307
[22]	validation_0-rmse:0.03292
[23]	validation_0-rmse:0.03292
[24]	validation_0-rmse:0.03307
[25]	validation_0-rmse:0.03317
[26]	validation_0-rmse:0.03375
[27]	validation_0-rmse:0.03381
[28]	validation_0-rmse:0.03380
[29]	validation_0-rmse:0.03386
[30]	validation_0-rmse:0.03399
[31]	validation_0-rmse:0.03374
[32]	validation_0-rmse:0.03373
[33]	validation_0-rmse:0.03368
[34]	validation_0-rmse:0.03381
[35]	validation_0-rmse:0.03418
[36]	validation_0-rmse:0.03400
[37]	validati

C:\Users\ruben\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\xgboost\callback.py:386: UserWarning: [16:37:48] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  self.starting_round = model.num_boosted_rounds()


[5]	validation_0-rmse:0.05342
[6]	validation_0-rmse:0.05351
[7]	validation_0-rmse:0.05366
[8]	validation_0-rmse:0.05374
[9]	validation_0-rmse:0.05377
[10]	validation_0-rmse:0.05387
[11]	validation_0-rmse:0.05303
[12]	validation_0-rmse:0.05381
[13]	validation_0-rmse:0.05380
[14]	validation_0-rmse:0.05434
[15]	validation_0-rmse:0.05456
[16]	validation_0-rmse:0.05458
[17]	validation_0-rmse:0.05454
[18]	validation_0-rmse:0.05467
[19]	validation_0-rmse:0.05465
[20]	validation_0-rmse:0.05521
[21]	validation_0-rmse:0.05553
[22]	validation_0-rmse:0.05572
[23]	validation_0-rmse:0.05608
[24]	validation_0-rmse:0.05579
[25]	validation_0-rmse:0.05585
[26]	validation_0-rmse:0.05602
[27]	validation_0-rmse:0.05595
[28]	validation_0-rmse:0.05616
[29]	validation_0-rmse:0.05608
[30]	validation_0-rmse:0.05607
[31]	validation_0-rmse:0.05707
[32]	validation_0-rmse:0.05708
[33]	validation_0-rmse:0.05694
[34]	validation_0-rmse:0.05704
[35]	validation_0-rmse:0.05703
[36]	validation_0-rmse:0.05707
[37]	validati

C:\Users\ruben\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\xgboost\sklearn.py:1028: UserWarning: [16:37:50] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  self.get_booster().save_model(fname)
C:\Users\ruben\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\xgboost\sklearn.py:1028: UserWarning: [16:37:51] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  self.get_booster().save_model(fname)
C:\Users\ruben\AppData\Local\Packages\PythonSoftwareFoun

✅ 424 modelos guardados


In [11]:
# --- Main Prediction Function (Called by the Server) ---
def predict(
    test: pl.DataFrame,
    label_lags_1_batch: pl.DataFrame,
    label_lags_2_batch: pl.DataFrame,
    label_lags_3_batch: pl.DataFrame,
    label_lags_4_batch: pl.DataFrame,
) -> pl.DataFrame:
    """
    This function is called by the API to generate predictions for the current time step.
    """
    # Convert Polars DataFrames to Pandas for easier manipulation
    test_df = test.to_pandas()

    # FIX: Ensure all feature columns are numeric, coercing errors to NaN
    feature_cols = [col for col in test_df.columns if col != 'date_id']
    for col in feature_cols:
        test_df[col] = pd.to_numeric(test_df[col], errors='coerce')

    # 1. Create targeted features from the raw price data
    test_df_featured = create_targeted_features(test_df, pairs_df)

    # 2. Add lagged target features from the API
    for i in range(1, 5):
        lag_df = locals()[f'label_lags_{i}_batch'].to_pandas()
        lag_df = lag_df.rename(columns={f'target_{j}': f'target_{j}_lag_{i}' for j in range(NUM_TARGET_COLUMNS)})
        
        # FIX: Explicitly select columns to merge to avoid duplicate 'label_date_id'
        cols_to_merge = [col for col in lag_df.columns if col.startswith('target_') or col == 'date_id']
        test_df_featured = pd.merge(test_df_featured, lag_df[cols_to_merge], on='date_id', how='left')

    # 3. Generate predictions for all 424 targets
    predictions = {}
    for i in range(NUM_TARGET_COLUMNS):
        current_target = f'target_{i}'
        model = trained_models[current_target]
        
        # Get the feature names the model was trained on
        model_features = model.feature_names_in_
        
        # Ensure all required features are present, fill missing with 0
        X_test = test_df_featured.copy()
        for col in model_features:
            if col not in X_test.columns:
                X_test[col] = 0 # Fill missing features (e.g., lags on first day) with a neutral value
        
        # Make prediction
        prediction = model.predict(X_test[model_features])
        predictions[current_target] = prediction[0]

    # 4. Format and return the predictions as a Polars DataFrame
    predictions_df = pl.DataFrame(predictions)
    
    assert isinstance(predictions_df, (pd.DataFrame, pl.DataFrame))
    assert len(predictions_df) == 1
    return predictions_df
# --- End of Script ---

In [12]:

test_features = pl.read_csv("mitsui-commodity-prediction-challenge/test.csv")
lag1 = pl.read_csv("mitsui-commodity-prediction-challenge/lagged_test_labels/test_labels_lag_1.csv")
lag2 = pl.read_csv("mitsui-commodity-prediction-challenge/lagged_test_labels/test_labels_lag_2.csv")
lag3 = pl.read_csv("mitsui-commodity-prediction-challenge/lagged_test_labels/test_labels_lag_3.csv")
lag4 = pl.read_csv("mitsui-commodity-prediction-challenge/lagged_test_labels/test_labels_lag_4.csv")
prediccionfinal = predict(test_features,
                            lag1,
                            lag2,
                            lag3,
                            lag4)

prediccionfinal


C:\Users\ruben\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\xgboost\core.py:729: UserWarning: [16:50:06] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)


target_0,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,target_9,target_10,target_11,target_12,target_13,target_14,target_15,target_16,target_17,target_18,target_19,target_20,target_21,target_22,target_23,target_24,target_25,target_26,target_27,target_28,target_29,target_30,target_31,target_32,target_33,target_34,target_35,target_36,…,target_387,target_388,target_389,target_390,target_391,target_392,target_393,target_394,target_395,target_396,target_397,target_398,target_399,target_400,target_401,target_402,target_403,target_404,target_405,target_406,target_407,target_408,target_409,target_410,target_411,target_412,target_413,target_414,target_415,target_416,target_417,target_418,target_419,target_420,target_421,target_422,target_423
f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
0.000765,-0.003322,0.000153,-0.00095,-0.002946,0.000102,0.000712,-0.000106,0.00331,-0.000392,-0.000049,-0.000822,0.000429,-0.002539,0.000633,0.000098,-0.001694,0.002655,0.000081,0.001102,0.000183,0.000353,0.000109,-0.00128,0.000776,-0.000287,-0.000721,0.00015,0.000461,-0.000134,-0.010114,-0.000238,-0.000721,0.000734,0.000086,-0.000926,-0.000591,…,0.009479,-0.000516,-0.000039,0.00103,-0.003805,0.00199,0.014117,-0.021241,-0.000211,0.00231,0.006765,-0.0015,0.000313,-0.000545,-0.000264,0.000289,-0.004464,-0.003837,-0.003222,-0.002078,0.000471,0.002525,-0.006868,-0.000354,-0.002159,0.001848,-0.002231,0.010635,0.001537,-0.003027,0.004756,-0.005197,-0.001625,0.006444,0.006513,-0.005084,-0.005935
